In [5]:
import keras
from keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing import image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
training_data = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
training_data = training_data.loc[:10_000]

training_data

In [ ]:
training_tags = set()
for tags in training_data.tags.unique():
    for i in tags.split():
        training_tags.add(i)

training_tags

In [ ]:
for name in training_tags:
    training_data[name] = 0

training_data

In [ ]:
for j in training_tags:
    for n, v in enumerate(training_data.tags.str.split()):
        if j in v:
            training_data.loc[n, j] = 1


In [ ]:
training_data

In [ ]:
image_path = '../input/planets-dataset/planet/planet/train-jpg/'
images = []

In [ ]:
for i in range(training_data.shape[0]):
    img = image.load_img(image_path + training_data['image_name'][i]+'.jpg', target_size=(140,180,3))
    img = image.img_to_array(img)
    img = img/255.0
    images.append(img)


x = np.array(images)
x = np.asarray(x).astype(np.float32)

In [ ]:
y = np.array(training_data.drop(['image_name', 'tags'], axis=1))
y = np.asarray(y).astype(np.float32)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=3, test_size=0.2)

In [ ]:
def fbeta(y_actual, y_pred, beta=2, epsilon=1e-4):
    beta_sq = beta**2
    y_actual = tf.cast(y_actual, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    TP = tf.reduce_sum(y_actual * y_pred, axis=1)
    FP = tf.reduce_sum(y_pred, axis=1) - TP
    FN = tf.reduce_sum(y_actual, axis=1) - TP
    
    precision = TP / (TP + FP + epsilon)
    recall = TP / (TP + FN + epsilon)
    
    fbeta_score = ((1 + beta_sq) * precision * recall) / (beta_sq * precision + recall + epsilon)
    
    return fbeta_score


## Building the CNN

In [ ]:
cnn = Sequential()

In [ ]:
# Convolution layer

cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(140,180,3)))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.2))

In [ ]:
# Second layer

cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.2))

In [ ]:
# Flattening

cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.2))

In [3]:
# Output layer

cnn.add(Dense(17, activation='sigmoid'))

NameError: name 'cnn' is not defined

In [ ]:
# Compiling the CNN

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=[fbeta])

## Training the Neural Network

In [ ]:
cnn.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

In [ ]:
# CNN evaluation

f_beta = cnn.evaluate(x_test, y_test)

print(f'Fbeta score is {fbeta}')